# RNN (Pytorch 첫걸음)

일반적인 신경망과 다르게, 내부 상태를 저장하고 있다. 특정 시점 t의 입력값 x(t)와 이전 시점의 내부상태 h(t-1)을 입력하면 새로운 내부상태 h(t)를 출력.

일반 신경망보다 train이 어렵다. 오랜 시간 쌓인 이력을 사용한다는 건 그만큼 layer가 deep하다는 것. gradient vanishing이나 경사 분실 문제 발생 가능.


따라서 RNN의 layer를 단순 선형(누적형태) 대신 정교한 처리 모듈로 변경한 LSTM이나 GRU 등의 RNN 모듈도 있다.

## 텍스트 데이터의 수치화

세 단계로 구성
1. 정규화 & 토큰화
2. Dictionary 구축
3. 수치로 변환

1. 문장을 특정 단위의 리스트로 분해한다. 예컨대 단어나 문자 등을 분할 단위로 사용. 유럽계 언어는 공백으로 구분해도 충분하지만 일본어, 중국어 등은 형태소 분석 처리가 필요하기도 함. 표기 차이도 통일해야 함. 예컨대 소문자로 전부 통일 / ~한다, ~하다 = ~하다로 통일. isn't 는 is not으로 통일하는 등.

2. 모든 문장의 집합(Corpus)에 대한 토큰을 수집하고, 숫자 id를 부여하는 작업. 등장 순서대로 부여하거나 빈도수에 따라 부여하거나.

3. 토큰의 리스트로 분할된 문장을 Dictionary를 사용해 id list로 변환한다.

이 작업을 거치며 하나의 긴 문자열이었던 문장이 '수치 리스트'로 변한다. 이 리스트를 다시 집계하고 id의 등장횟수를 벡터로 표현한 것이 BoW.

ex) (I, you, am, of ...) = (1,0,1,3 ..)

BoW는 계산이 간단하고, 여러 문장을 모으면 희소행렬로 표현할 수 있어 효율은 좋다. 단 토큰 순서 정보를 잃는다.

신경망에서는 Embedding이라는 기법으로 토큰을 벡터화하고, 벡터 데이터의 시계열로 문장을 처리하는 것이 주류임.

Pytorch는 nn.Embedding으로 layer 생성이 가능하다.


```python
# 전체 10,000 종류의 토큰을 20차원 벡터로 표현하는 경우
emb = nn.Embedding(10000, 20, padding_idx = 0)
# Embedding layer input타입은 int64
inp = torch.tensor([1,2,5,2,10], dtype = torch.int64)
# 출력은 float32
out = emb(inp)
```

padding_idx를 지정하므로, 이 경우 id가 0인 벡터로 바뀐다. 사전에 없는 토큰은 id가 0, 실제 id는 1부터 시작하도록 세팅한다.

토큰 종류는 0을 포함한 수를 nn.Embedding의 첫 번째 인수로 지정해야 한다.


(nn.Embedding 값도 미분 가능. 내부의 가중치 파라미터의 학습이 가능하다는 의미다. Neural Net 학습 시 여기도 최적화가 가능함. 사전에 학습된 nn.Embedding 값에 기반한 transfer Learning 가능)  -- ex) Word2Vec으로 유명한 Continuous-BOW나 Skip-Gram 등의 모델을 쓰는 경우

## RNN 문장분류

In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xf aclImdb_v1.tar.gz

--2019-05-20 06:10:00--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: 'aclImdb_v1.tar.gz'

aclImdb_v1.tar.gz   100%[===================>]  80.23M  10.1MB/s    in 9.4s    

2019-05-20 06:10:13 (8.54 MB/s) - 'aclImdb_v1.tar.gz' saved [84125825/84125825]



In [3]:
import glob
import pathlib
import re

In [5]:
remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

In [6]:
def text_to_ids(text, vocab_dict):
    # ?! 이외의 기호 삭제
    text = remove_marks_regex.sub("",text)
    # ?!와 단어 사이의 공백 삽입
    text = shift_marks_regex.sub(r" \1 ",text)
    tokens = text.split()
    return [vocab_dict.get(token,0) for token in tokens]
# 긴 문자열을 토큰 ID 리스트로 변환하는 함수. 정규식으로 문장부호나 괄호 제거, ?나 ! 사이에 공백을 넣어 단어와 별도의 토큰으로 분할한 것.
# ?, !가 imdb.vocab이 포함되어 있기 때문. 용어집에 없는 토큰은 값을 0으로 할당한다.

In [7]:
def list_to_tensor(token_idxes, max_len=100, padding=True):
    if len(token_idxes) > max_len:
        token_idxes = token_idxes[:max_len]
    n_tokens = len(token_idxes)
    if padding:
        token_idxes = token_idxes + [0] * (max_len - len(token_idxes))
    return torch.tensor(token_idxes, dtype=torch.int64), n_tokens
# id 리스트를 int64 텐서로 변환하는 함수. 변환할 때는 각 문장을 분할한 후 토큰 수를 제한, 그 수에 미치지 못하는 경우에는 id 0 할당한다.

### Dataset 클래스 작성

생성자 내에서 텍스트 파일의 경로와 레이블을 모은 튜플 리스트 작성, __getitem__ 내에서 이 파일을 읽어 텐서로 변환한다.

텐서는 max_len으로 지정한 길이로 통일. (padding) -> 나중에 처리하기 용이하다. + 0으로 채우기 전 원래 길이와 n_tokens도 이후에 필요하므로 함께 반환한다.

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
!conda install -c conda-forge tqdm
!y